In [41]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
import pandas as pd
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
df_task1_test = pd.read_csv('test.csv')

df_task1_test.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price
0,1029166,BMW X6 xDrive35i Sunroof (New 10-yr COE),bmw,x6,"owner consignment unit, viewing strictly by ap...",2012.0,NaN,27-jun-2012,suv,coe car,auto,2070.0,225.0,NaN,2979.0,3.0,14870.0,NaN,2362.0,NaN,122000.0,68994.0,68994.0,NaN,NaN,uncategorized,"smooth inline 6 3.0l turbo n55 engine, high sp...","20'' staggered m rims, carbon steering wheel, ...",NaN
1,1017714,Porsche 911 Carrera S Coupe 3.8A PDK (COE till...,porsche,911,the 911 carrera s displacing 3.8 litres with m...,2010.0,NaN,11-may-2010,sports car,"coe car, direct owner sale",auto,1530.0,283.0,NaN,3800.0,5.0,31920.0,31553.0,4366.0,23211.0,125000.0,115405.0,115405.0,NaN,NaN,uncategorized,responsive flat six 3.8 litres of naturally as...,"pasm, sports chrono, premium red leather packa...",NaN
2,1005265,Porsche Macan Diesel S 3.0A PDK,porsche,macan,comes with agent warranty till january 2022. a...,2016.0,NaN,18-jan-2017,suv,"parf car, premium ad car",auto,1955.0,190.0,diesel,2967.0,3.0,26640.0,53106.0,3336.0,102072.0,55965.0,69922.0,97860.0,NaN,NaN,uncategorized,black exterior with black interior. comes with...,"keyless entry/start, reverse camera, navi, ldw...",NaN
3,1029464,BMW 5 Series 530i Luxury,bmw,530i,a careful owner upgraded to a porsche macan. t...,2017.0,NaN,28-jun-2017,luxury sedan,"parf car, sgcarmart warranty cars",auto,1540.0,185.0,NaN,1998.0,1.0,19390.0,47501.0,1210.0,73330.0,100000.0,49363.0,61109.0,NaN,NaN,uncategorized,"2.0l bmw inline 4 cylinder twin power turbo, 2...","smart key, navigation, keyless entry, push sta...",NaN
4,1017727,Honda Vezel 1.5A X,NaN,vezel,100% non phv-unit! excellent condition and wel...,2016.0,NaN,05-jun-2017,suv,parf car,auto,1190.0,96.0,NaN,1496.0,2.0,9620.0,46489.0,682.0,34555.0,62000.0,20184.0,10258.0,NaN,NaN,uncategorized,"1.5l dohc i-vtec engine, 128 bhp, cvt auto tra...","sports rims, leather seats, andriod player, re...",NaN


In [46]:
listing_ids = list(df_task1_test['listing_id'].unique())
print(len(listing_ids))

4995


In [47]:
from lxml import html
import requests
import concurrent.futures
import time
import csv


listing_id_price_map = {}

MAX_THREADS = 30

def put_price_in_map_by_listing_id(listing_id):
    # Request the page
    page = requests.get('https://www.sgcarmart.com/used_cars/info.php?ID=' + str(listing_id), verify=False)
    tree = html.fromstring(page.content)  

    # Get element using XPath
    buyers = tree.xpath('/html/body/div[2]/div/div[3]/div[1]/div/div/a')

    buyers = tree.xpath('/html/body/div[2]/div/div[3]/div[5]/div[1]/div[1]/table/tr[1]/td[2]/a/strong/text()')
    if len(buyers) == 0:
        listing_id_price_map[listing_id] = np.nan
    else:
        price = float(buyers[0].replace('$', '').replace(',', ''))
        listing_id_price_map[listing_id] = price
    return True

put_price_in_map_by_listing_id(listing_ids[0])


threads = min(MAX_THREADS, len(listing_ids))

t0 = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(put_price_in_map_by_listing_id, listing_ids)
t1 = time.time()
print(f"{t1-t0} seconds to download {len(listing_ids)} listings.")

with open('test_price.csv', 'w') as f:
    f.write("%s,%s\n"%("listing_id", "price"))
    for key in listing_id_price_map.keys():
        f.write("%s,%s\n"%(key,listing_id_price_map[key]))

564.5824320316315 seconds to download 4995 listings.


In [48]:
len(listing_id_price_map)

4995

In [49]:
df_task1_test_price = pd.read_csv('test_price.csv')

df_task1_test_price.head()

,listing_id,price
0,1011450,395000.0
1,1013897,53588.0
2,1028794,79700.0
3,767227,NaN
4,1028562,NaN


In [50]:
df_task1_test_price.dropna(subset=['price'], inplace = True)
len(df_task1_test_price)

1140